In [ ]:
from slim_gsgp_lib_np.config.gp_config import *
# from slim_gsgp_lib_np.algorithms.GP.operators.crossover_operators import *
# from slim_gsgp_lib_np.algorithms.GP.operators.mutators import * 

from slim_gsgp_lib_np.algorithms.GP.representations.tree_utils import create_full_random_tree, create_grow_random_tree, create_neutral_tree

from slim_gsgp_lib_np.datasets.synthetic_datasets import * 
from slim_gsgp_lib_np.main_gp import gp
from slim_gsgp_lib_np.algorithms.GP.representations.tree_utils import flatten

from functions.utils_test import simplify_tuple_expression 

from matplotlib import pyplot as plt
import numpy as np
import random
from slim_gsgp_lib_np.algorithms.GP.representations.tree_utils import (random_subtree, substitute_subtree, 
                                                                       get_indices_with_levels, get_depth)
from slim_gsgp_lib_np.utils.utils import swap_sub_tree, get_subtree    
from collections import defaultdict

In [2]:
X, y, _, _ = load_synthetic1()
tree, pop = gp(X_train=X, y_train=y, test_elite=False, verbose=False, pop_size=10, n_iter=10, dataset_name='test', full_return=True)
FUNCTIONS = tree.FUNCTIONS
TERMINALS = tree.TERMINALS
CONSTANTS = tree.CONSTANTS

In [30]:
sample1 = create_grow_random_tree(5, FUNCTIONS, TERMINALS, CONSTANTS, p_c=0.2, p_t=0.5, p_cond=0.5)
simplify_tuple_expression(sample1, False)

Piecewise((x0, x2 > 0), (x2, True))

In [31]:
sample1

('cond', 'x2', 'x0', 'x2')

In [37]:
sample2 = create_grow_random_tree(5, FUNCTIONS, TERMINALS, CONSTANTS, p_c=0.2, p_t=0.5, p_cond=0.5)
simplify_tuple_expression(sample2, False)

Piecewise((x2, x2*(x0 + x1) > 0), (x0, True))

In [38]:
sample2

('cond', ('multiply', ('add', 'x0', 'x1'), 'x2'), 'x2', 'x0')

In [ ]:
def crossover_trees(max_depth,
                    **kwargs):
    
    def inner_xo(tree1, tree2):
        
        indices_with_levels_tree1 = get_indices_with_levels(tree1.repr_)
        indices_with_levels_tree2 = get_indices_with_levels(tree2.repr_)

        level1 = random.choice(list(indices_with_levels_tree1.keys()))
        index1 = random.choice(indices_with_levels_tree1[level1])
        subtree1 = get_subtree(tree1.repr_, list(index1))
        depth1 = get_depth(subtree1)

        max_level_2 = min(max_depth - depth1, tree2.depth - 1)

        while True:
            level2 = random.choice(range(0, max_level_2 + 1))
            index2 = random.choice(indices_with_levels_tree2[level2])
            subtree2 = get_subtree(tree2.repr_, list(index2))
            depth2 = get_depth(subtree2)

            if depth2 <= max_depth - level1 and (level2 > 0 or depth1 > 1) and (level1 > 0 or depth2 > 1):
                break

        # Swap the subtrees
        new_tree1 = swap_sub_tree(tree1.repr_, subtree2, list(index1))
        new_tree2 = swap_sub_tree(tree2.repr_, subtree1, list(index2))
        return new_tree1, new_tree2

    return inner_xo

In [286]:
def get_blocks_with_levels(tree, root=False):
    indices_by_level = defaultdict(list)

    def traverse(sub_tree, path=(), level=0):
        if not isinstance(sub_tree, tuple):
            indices_by_level[level].append(path)
        else:
            if path != ():  # don't include the root twice
                indices_by_level[level].append(path)
            op, *args = sub_tree
            if op == 'cond' : 
                for i, child in enumerate(args[1:], start=1):
                    traverse(child, path + (i + 1,), level + 1) 
            
            else: 
                for i, child in enumerate(args):
                    traverse(child, path + (i + 1,), level + 1)

    traverse(tree)
    indices_by_level[0].append(()) if root else None 

    return dict(indices_by_level)

RULES :
(1) Can't swap predicates 
(2) Can't swap condition blocks with functions, except if the condition block is the last and the function is at the root 

In [58]:
sample2

('cond', ('multiply', ('add', 'x0', 'x1'), 'x2'), 'x2', 'x0')

In [110]:
sample3 = create_grow_random_tree(6, FUNCTIONS, TERMINALS, CONSTANTS, p_c=0.2, p_t=0.5, p_cond=0.5)
simplify_tuple_expression(sample3, False)
idl3 = get_indices_with_levels(sample3)

Piecewise((Piecewise((-1.25*x0/(x1*(x1 - 0.806225774829855)), x2 > 0), (x0, True)), x0 > 0), (Piecewise((x2, x2*(x0 + x1) > 0), (x0, True)), True))

In [211]:
idl1 = get_indices_with_levels(sample1)
idl2 = get_indices_with_levels(sample2)
has_cond1 = len(idl1[1]) == 3
has_cond2 = len(idl2[1]) == 3

print(idl1, '\n\n', idl2)

{1: [(1,), (2,), (3,)], 2: [(2, 1), (2, 2), (2, 3), (3, 1), (3, 2), (3, 3)], 3: [(2, 2, 1), (2, 2, 2), (3, 1, 1), (3, 1, 2)], 4: [(2, 2, 1, 1), (2, 2, 1, 2), (2, 2, 2, 1), (2, 2, 2, 2), (3, 1, 1, 1), (3, 1, 1, 2)], 5: [(2, 2, 1, 2, 1), (2, 2, 1, 2, 2), (2, 2, 2, 1, 1)], 0: [()]} 

 {1: [(1,), (2,), (3,)], 2: [(1, 1), (1, 2)], 3: [(1, 1, 1), (1, 1, 2)], 0: [()]}


In [112]:
sample3

('cond',
 'x0',
 ('cond',
  'x2',
  ('divide',
   ('divide', 'x0', ('multiply', 'constant_0.8', 'x1')),
   ('subtract', ('sqrt', 'constant_-0.65'), 'x1')),
  'x0'),
 ('cond', ('multiply', ('add', 'x0', 'x1'), 'x2'), 'x2', 'x0'))

In [207]:
ilev = get_indices_with_levels(sample3)
l1 = random.choice(list(ilev.keys()))
i1 = list(random.choice(ilev[l1]))
st1 = get_subtree(sample3, i1)

In [213]:
ilev1 = get_indices_with_levels(sample1)
ilev2 = get_indices_with_levels(sample2)
l1 = random.choice(list(ilev1.keys()))
i1 = list(random.choice(ilev1[l1]))
st1 = get_subtree(sample1, i1)

In [229]:
sample1[2]

('cond',
 'x2',
 ('divide',
  ('divide', 'x0', ('multiply', 'constant_0.8', 'x1')),
  ('subtract', ('sqrt', 'constant_-0.65'), 'x1')),
 'x0')

In [218]:
get_blocks_with_levels(sample1)

{1: [(2,), (3,)],
 2: [(2, 2), (2, 3), (3, 2), (3, 3)],
 3: [(2, 2, 1), (2, 2, 2)],
 4: [(2, 2, 1, 1), (2, 2, 1, 2), (2, 2, 2, 1), (2, 2, 2, 2)],
 5: [(2, 2, 1, 2, 1), (2, 2, 1, 2, 2), (2, 2, 2, 1, 1)]}

In [ ]:
md = 5
xo = crossover_trees(max_depth=md)


if has_cond1 and has_cond2:
    pass

elif not has_cond1 and not has_cond2:
    trees = xo(sample1, sample2)

elif not has_cond1: 
    bl = get_blocks_with_levels(sample2)
    l2 = random.choice(list(bl.keys()))
    i2 = list(random.choice(bl[l2]))
    st2 = get_subtree(sample2, i2)

    if st2[0] == 'cond': 
        st1 = sample1

    else: 
        l1 = random.choice(list(idl1.keys()))
        i1 = random.choice(idl1[l1])
        st1 = get_subtree(sample1, i1)

elif not has_cond2: 
    bl = get_blocks_with_levels(sample1)
    l1 = random.choice(list(bl.keys()))
    i1 = list(random.choice(bl[l1]))
    st1 = get_subtree(sample1, i1)
    
    if st1[0] == 'cond': 
        st2 = sample2

    else: 
        l2 = random.choice(list(idl2.keys()))
        i2 = random.choice(idl2[l2])
        st2 = get_subtree(sample2, i2) 

In [ ]:
# IF NO CONDITIONS 
trees = xo(sample1, sample2)

In [289]:
get_blocks_with_levels(sample1, root=True)

{1: [(2,), (3,)],
 2: [(2, 2), (2, 3), (3, 2), (3, 3)],
 3: [(2, 2, 1), (2, 2, 2)],
 4: [(2, 2, 1, 1), (2, 2, 1, 2), (2, 2, 2, 1), (2, 2, 2, 2)],
 5: [(2, 2, 1, 2, 1), (2, 2, 1, 2, 2), (2, 2, 2, 1, 1)],
 0: [()]}

In [297]:
# IF BOTH HAVE CONDITIONS
bl = get_blocks_with_levels(sample1, root=True)
l1 = random.choice(list(bl.keys()))
id1 = list(random.choice(bl[l1]))
st1 = get_subtree(sample1, id1)

# IF CONDITION : 
if st1[0] == 'cond': 
    if l1 == 0: 
        pass

    else: 
        pass


# IF NOT CONDITION : can never be swapped with a condition
l1

3

In [ ]:
def tree_depth(tree):
    """Compute max depth of nested tree."""
    if not isinstance(tree, (tuple, list)):
        return 0
    return 1 + max((tree_depth(child) for child in tree[1:]), default=0)

def is_condition(node):
    return isinstance(node, (tuple, list)) and node and node[0] == 'cond'

def contains_condition(node):
    """True if any descendant is a cond node."""
    if is_condition(node):
        return True
    if not isinstance(node, (tuple, list)):
        return False
    return any(contains_condition(child) for child in node)


def crossover_trees(max_depth):
    """
    Returns a function xo(ind1, ind2) performing one-point crossover
    while respecting piecewise condition rules and max depth.
    """
    def xo(ind1, ind2):
        # identify presence of cond in each
        def has_cond(tree):
            return contains_condition(tree)
        has1, has2 = has_cond(ind1), has_cond(ind2)

        # simple symmetric case: both or neither have cond
        if (has1 and has2) or (not has1 and not has2):
            return _two_way_crossover(ind1, ind2)

        # one has cond, the other doesn't
        # sample1 lacks cond -> sample1 is ind1, sample2 is ind2
        if not has1:
            return _one_cond_cross(ind1, ind2)
        else:
            # swap roles
            off2, off1 = _one_cond_cross(ind2, ind1)
            return off1, off2

    def _two_way_crossover(t1, t2):
        # swap any matching-level blocks excluding predicate slots
        b1 = get_blocks_with_levels(t1)
        b2 = get_blocks_with_levels(t2)
        candidates = []
        for lvl in b1:
            if lvl in b2:
                for i1 in b1[lvl]:
                    # skip predicate positions
                    if i1[-1] == 1 and is_condition(get_subtree(t1, i1[:-1])):
                        continue
                    for i2 in b2[lvl]:
                        if i2[-1] == 1 and is_condition(get_subtree(t2, i2[:-1])):
                            continue
                        candidates.append((i1, i2))
        if not candidates:
            return t1, t2
        i1, i2 = random.choice(candidates)
        s1, s2 = get_subtree(t1, i1), get_subtree(t2, i2)
        o1 = swap_sub_tree(t1, s2, i1)
        o2 = swap_sub_tree(t2, s1, i2)
        if tree_depth(o1) <= max_depth and tree_depth(o2) <= max_depth:
            return o1, o2
        return t1, t2

    def _one_cond_cross(no_cond, has_cond_tree):
        # select a block from has_cond_tree
        blocks = get_blocks_with_levels(has_cond_tree)
        lvl = random.choice(list(blocks.keys()))
        idx2 = random.choice(blocks[lvl])
        sub2 = get_subtree(has_cond_tree, idx2)
        # if the picked subtree is a condition, leave the no_cond parent intact
        if is_condition(sub2):
            return no_cond, has_cond_tree
        # otherwise pick a non-predicate block from no_cond
        blocks1 = get_blocks_with_levels(no_cond)
        # filter out cond blocks in no_cond
        flat = [(l,i) for l in blocks1 for i in blocks1[l]
                if not is_condition(get_subtree(no_cond, i))]
        if not flat:
            return no_cond, has_cond_tree
        l1, idx1 = random.choice(flat)
        sub1 = get_subtree(no_cond, idx1)
        off1 = swap_sub_tree(no_cond, sub2, idx1)
        off2 = swap_sub_tree(has_cond_tree, sub1, idx2)
        if tree_depth(off1) <= max_depth and tree_depth(off2) <= max_depth:
            return off1, off2
        return no_cond, has_cond_tree

    return xo

In [299]:
crossover_trees(max_depth=5)(sample1, sample2)

IndexError: tuple index out of range